In [ ]:
# Standard imports
import os
import sys
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
sys.path.append(r"/mnt/data/the_deep/Geotech_Research/Critical_Soil_Models/pumat")

from lib.Load_Classes.Popular_Load_Class import PopularPath
from lib.Driver_Classes.Mod_Driver_Setup import DriverModelSetup
from lib.Driver_Classes.Mod_Driver_Model import DriverModel
from lib.general_functions.executing_runs import generate_batch_script, run_batch_script

In [ ]:
trx_strain = "OedometricE1"

load_params = {
    "ninc": 1000,
    "maxiter": 0,
    "dtime"  : 500_000,
    "every"  : 10,
    "ddstran_1": -0.4,
}


load = PopularPath(trx_strain, load_params)

In [ ]:
# This needs to be in the order that the parameters should be in

initial_fric_angle = 30.0
initial_cohesion = 0
newton_properties = {
    "Shear Modulus"          : 100,
    "poisson ratio"          : 0.2,
    "peak cohesion"          : initial_cohesion,
    "residual cohesion"      : 0,
    "peak fric angle"        : initial_fric_angle,
    "residual fric angle"    : 0,
    "peak dilatancy"         : 10, 
    "residual dilatancy"     : 0,
    "softening shape factor" : 0,
    "integration flag"       : 0, # zero for substepping, 1 for Ortiz-Simo
    "Yield surface tolerance": 1e-8,
    "num integration iters"  : 1000,
    "Euler_DT_min"           : 1e-8 # minimum pseudo time step for sloan integration
}

# Set the ortiz properties to be the same as the newton properties except change the integration flag to be 1
ortiz_properties = newton_properties.copy()
ortiz_properties["integration flag"] = 1

In [ ]:
# Get the name of the current folder
current_working_dir = os.getcwd()

# Path to the folder that the Ortiz-Simo data should be stored in
Ortiz_folder = os.path.join(current_working_dir, "Ortiz_Simo")

# Path to the folder that the Newton Raphson data should be stored in
Newton_folder = os.path.join(current_working_dir, "Newton_Raphson")

exe_path = r"/mnt/data/the_deep/Geotech_Research/Critical_Soil_Models/MohrCoulomb_StrainSoft/build/gfortran_E167FD2A985B468F/app/MCSS_incrementalDriver"


In [ ]:
# Generate the 
models = [DriverModel(Ortiz_folder, "Ortiz_MCSS", exe_path, "output.txt"),
          DriverModel(Newton_folder, "Newton_MCSS", exe_path, "output.txt")
]



In [ ]:
for model in models:
    model.setup.clear_folder()

In [ ]:
# Set the ortiz simo model
models[0].setup.write_parameters_file(ortiz_properties)

# Set the Newton model
models[1].setup.write_parameters_file(newton_properties)

In [ ]:
# Set the stress and state parameters
stress = [-10, -10, -10, 0, 0, 0]
state_params = {"cohesion": initial_cohesion,
                "fric angle": initial_fric_angle * np.pi/180,
                "dilatancy angle": 0,
                "EspP_1": 0,
                "EspP_2": 0,
                "EspP_3": 0,
                "EspP_4": 0,
                "EspP_5": 0,
                "EspP_6": 0
}

for model in models:
    model.setup.write_initial_conditions_file(stress, state_params)
    model.setup.store_loads(load)
    model.setup.write_loads()

In [ ]:
for model in models:
    model.run_model()

In [ ]:
for model in models:
    print(model.results)

In [ ]:
for model in models:
    model.results.store_all()

In [ ]:
# Generate a figure to plot the MCSS models
fig, axs = plt.subplots(nrows = 2, ncols = 2, figsize = (8, 8))

colors = ["blue", "red"]
labels = ["Ortiz-Simo", "Newton-Raphson"]
styles = ["dashed", "solid"]
line_widths = [4, 1.75]

for model, color, label, style, linewidth in zip(models, colors, labels, styles, line_widths):
    model.results.quick_quad_plot(axial_strain_id="stran(1)", compression_pos=True, axs = axs, 
                                  color = color , label=label, legend = True, linestyle = style, linewidth = linewidth)
    
